# MySQL-БД
## Подключение к базе

In [1]:
# !pip3 install mysql-connector-python
# !pip3 install mysql

import mysql.connector

In [2]:
with open('mysql_password.txt', 'r', encoding='utf-8') as f:
    pwd = f.read().strip()

con = mysql.connector.connect(host='127.0.0.1', port=3306, user='root', password=pwd)
cur = con.cursor(dictionary=True)

In [3]:
# cur.execute("""DROP SCHEMA IF EXISTS corpus""")
# con.commit()

## Схема

In [4]:
cur.execute("""
CREATE SCHEMA IF NOT EXISTS corpus DEFAULT CHARACTER SET utf8
""")

In [5]:
con = mysql.connector.connect(host='127.0.0.1', port=3306,
                              database='corpus',
                              user='root', password=pwd)
cur = con.cursor(dictionary=True, buffered=True)

## Таблицы

In [6]:
# тут разрешаем оставлять без автора
cur.execute("""
CREATE TABLE IF NOT EXISTS texts (
  id_texts INT NOT NULL,
  text VARCHAR(10000) NOT NULL,
  genre VARCHAR(128) NOT NULL,
  title VARCHAR(256) NOT NULL,
  author VARCHAR(256) NULL,
  PRIMARY KEY (id_texts),
  UNIQUE INDEX id_texts_UNIQUE (id_texts ASC) VISIBLE,
  INDEX title_INDEX (title ASC) VISIBLE);
""")

In [7]:
cur.execute("""
CREATE TABLE IF NOT EXISTS sentences (
  id_sentences INT NOT NULL,
  sent_text VARCHAR(1000) NOT NULL,
  sent_num_in_text INT NOT NULL,
  start INT NOT NULL,
  end INT NOT NULL,
  text_id INT NOT NULL,
  PRIMARY KEY (id_sentences),
  UNIQUE INDEX id_sentences_UNIQUE (id_sentences ASC) VISIBLE,
  INDEX text_id_INDEX (text_id ASC) VISIBLE,
  CONSTRAINT sent_to_texts
    FOREIGN KEY (text_id)
    REFERENCES texts (id_texts)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
""")

In [8]:
# разрешаем пустые сорсы и таргеты, т.к. у коллокаций их нет
cur.execute("""
CREATE TABLE IF NOT EXISTS metaphors (
  id_metaphors INT NOT NULL,
  meta_text VARCHAR(64) NOT NULL,
  meta_num_in_sent INT NOT NULL,
  start INT NOT NULL,
  end INT NOT NULL,
  source VARCHAR(128) NULL,
  target VARCHAR(128) NULL,
  is_collocation TINYINT NOT NULL,
  sent_id INT NOT NULL,
  text_id INT NOT NULL,
  PRIMARY KEY (id_metaphors),
  UNIQUE INDEX id_metaphors_UNIQUE (id_metaphors ASC) VISIBLE,
  INDEX sent_id_INDEX (sent_id ASC) VISIBLE,
  INDEX text_id_INDEX (text_id ASC) VISIBLE,
  INDEX source_INDEX (source ASC) VISIBLE,
  INDEX target_INDEX (target ASC) VISIBLE,
  CONSTRAINT meta_to_sent
    FOREIGN KEY (sent_id)
    REFERENCES sentences (id_sentences)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT meta_to_text
    FOREIGN KEY (text_id)
    REFERENCES texts (id_texts)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
""")

In [9]:
# здесь id токена делаем BIGINT
cur.execute("""
CREATE TABLE IF NOT EXISTS tokens (
  id_tokens BIGINT NOT NULL,
  tok_text VARCHAR(64) NOT NULL,
  tok_num_in_meta INT NULL,
  start INT NOT NULL,
  end INT NOT NULL,
  pos VARCHAR(64) NOT NULL,
  lemma VARCHAR(64) NOT NULL,
  deprel VARCHAR(64) NULL,
  head BIGINT NULL,
  meta_id INT NULL,
  sent_id INT NOT NULL,
  text_id INT NOT NULL,
  PRIMARY KEY (id_tokens),
  UNIQUE INDEX id_tokens_UNIQUE (id_tokens ASC) VISIBLE,
  INDEX meta_id_INDEX (meta_id ASC) VISIBLE,
  INDEX sent_id_INDEX (sent_id ASC) VISIBLE,
  INDEX text_id_INDEX (text_id ASC) VISIBLE,
  INDEX head_INDEX (head ASC) VISIBLE,
  CONSTRAINT tok_to_meta
    FOREIGN KEY (meta_id)
    REFERENCES metaphors (id_metaphors)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT tok_to_sent
    FOREIGN KEY (sent_id)
    REFERENCES sentences (id_sentences)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT tok_to_text
    FOREIGN KEY (text_id)
    REFERENCES texts (id_texts)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
""")

## DKPro-Cassis

In [10]:
# !pip install dkpro-cassis
from cassis import *

import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [11]:
folders = [os.path.join('annotation_xml', folder) for folder in os.listdir('annotation_xml')]
folders

['annotation_xml\\01_ Горький.conllu',
 'annotation_xml\\02_ Горький.conllu',
 'annotation_xml\\03_ Russian Love Story.conllu',
 'annotation_xml\\04_ Russian Love Story.conllu',
 'annotation_xml\\05_ Вестник района.conllu',
 'annotation_xml\\06_ Вестник района.conllu',
 'annotation_xml\\07_ научная статья.conllu',
 'annotation_xml\\08_ научная статья.conllu',
 'annotation_xml\\09_ Википедия.conllu',
 'annotation_xml\\10_ Википедия.conllu']

In [12]:
def get_is_metaphor(cas, text_id):

    # собираем спаны метафор
    starts, ends, meta_texts = [], [], []
    for segment in cas.select(f'webanno.custom.Is_metaphor_Masha'):
        starts.append(segment.begin)
        ends.append(segment.end)
        meta_texts.append(segment.get_covered_text())

    collocs = pd.DataFrame()
    collocs['starts'] = starts
    collocs['ends'] = ends
    collocs['text'] = meta_texts
    collocs['text_id'] = text_id

    for name in ['Masha', 'Liza', 'Olesia']:
        collocs[f"is_metaphor_{name}"] = [
            segment.is_metaphor
            for segment in cas.select(f'webanno.custom.Is_metaphor_{name}')]
    return collocs.to_dict(orient='records')

In [13]:
def get_sources_targets(cas, text_id):

    starts, ends, meta_texts = [], [], []
    ends = []
    meta_texts = []

    for segment in cas.select('webanno.custom.Collocation'):
        starts.append(segment.begin) 
        ends.append(segment.end)
        meta_texts.append(segment.get_covered_text())

    collocs = pd.DataFrame()
    collocs['starts'] = starts
    collocs['ends'] = ends
    collocs['text'] = meta_texts
    collocs['text_id'] = text_id

    for name in ['Masha', 'Liza', 'Olesia']:
        sources, targets = [], []

        for segment in cas.select(f'webanno.custom.Collocation'): 
            sources.append(segment[f'source_{name}'])
            targets.append(segment[f'target_{name}'])

        collocs[f"source_{name}"] = sources
        collocs[f"target_{name}"] = targets

    return collocs.to_dict(orient='records')

In [14]:
genres = ['Thematic news', 'Thematic news',
          'Fiction', 'Fiction',
          'Science', 'Science',
          'General news', 'General news',
          'Wikipedia', 'Wikipedia']

In [15]:
texts = []
sentences = []
collocations = []
is_metaphor = []
sources = []
tokens = []

for i, folder in enumerate(folders):
    text_id = i + 1
    path_to_typesystem = os.path.join(folder, 'demo\TypeSystem.xml')
    path_to_cas = os.path.join(folder, 'demo\demo.xmi')
    with open(path_to_typesystem, 'rb') as f:
        typesystem = load_typesystem(f)
    with open(path_to_cas, 'rb') as f:
        cas = load_cas_from_xmi(f, typesystem=typesystem)

    # данные о тексте
    doc_meta = cas.select('de.tudarmstadt.ukp.dkpro.core.api.metadata.type.DocumentMetaData')[0]
    texts.append({'text_id': text_id,
                  'text': doc_meta.get_covered_text(),
                  'genre': genres[i],
                  'title': doc_meta.documentTitle,
                  'author': None})
    # данные о предложениях
    for segment in cas.select('de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence'):
        sent_num = segment.id
        sentences.append({'text_id': text_id,
                          'sent_id': int(f'{text_id}00{sent_num}'),
                          'sent_num_in_text': sent_num,
                          'starts': segment.begin,
                          'ends': segment.end,
                          'text': segment.get_covered_text()})
    # данные о коллокациях
    for colloc in cas.select('custom.Span'):
        collocations.append({'text_id': text_id,
                             'starts': colloc.begin,
                             'ends': colloc.end,
                             'is_collocation': colloc.is_collocation,
                             'text': colloc.get_covered_text()})
    # данные о метафоричности
    is_metaphor.extend(get_is_metaphor(cas, text_id))

    # данные о source/target
    sources.extend(get_sources_targets(cas, text_id))

    # данные о токенах
    for token in cas.select('de.tudarmstadt.ukp.dkpro.core.api.syntax.type.dependency.Dependency'):
        tokens.append({'text_id': text_id,
                       'starts': token.begin,
                       'ends': token.end,
                       'text': token.Dependent.get_covered_text(),
                       'lemma': token.Dependent.lemma.value,
                       'pos': token.Dependent.pos.coarseValue,
                       'deprel': token.DependencyType,
                       'head_starts': token.Governor.begin,
                       'head_ends': token.Governor.end})

In [16]:
len(tokens)

12183

### Тексты

In [17]:
text_df = pd.DataFrame(texts)
text_df

,text_id,text,genre,title,author
0,1,Издательство « Ад Маргинем » выпустило недавно...,Thematic news,01_%20Горький.conllu,None
1,2,Библейские мотивы в романе из советской школьн...,Thematic news,02_%20Горький.conllu,None
2,3,Знак 2 .\nВстреча .\nВстретились мы с ней скор...,Fiction,03_%20Russian%20Love%20Story.conllu,None
3,4,"Знак 7 .\nМолочный шоколад Из всех знаков , эт...",Fiction,04_%20Russian%20Love%20Story.conllu,None
4,5,Руководители сельхозпредприятий за все в ответ...,Science,05_%20Вестник%20района.conllu,None
5,6,Азбука безопасности Будьте осторожны на льду !...,Science,06_%20Вестник%20района.conllu,None
6,7,1 .\nВведение Развитие новых квантовых приборо...,General news,07_%20научная%20статья.conllu,None
7,8,1 .\nВведение За последнее десятилетие произош...,General news,08_%20научная%20статья.conllu,None
8,9,"Градищанские хорваты ( также "" бургенландские ...",Wikipedia,09_%20Википедия.conllu,None
9,10,Ратенов ( ) — пассажирская станция в городе Ра...,Wikipedia,10_%20Википедия.conllu,None


### Предложения

In [18]:
sent_df = pd.DataFrame(sentences)
print(sent_df.shape)
sent_df.head()

(685, 6)


,text_id,sent_id,sent_num_in_text,starts,ends,text
0,1,1000,0,0,126,Издательство « Ад Маргинем » выпустило недавно...
1,1,1001,1,127,250,По просьбе « Горького » об этой книге рассказы...
2,1,1002,2,251,393,Недавно вышедший в издательстве « Ад Маргинем ...
3,1,1003,3,394,662,"В наших краях этот автор почти неизвестен , че..."
4,1,1004,4,663,1016,"В Европе Танпынар тоже весьма популярен , но п..."


### Метафоры

In [19]:
def merge_texts(text1, text2, text3):
    if not pd.isnull(text1):
        return text1
    if not pd.isnull(text2):
        return text2
    return text3

In [20]:
coll_df = pd.DataFrame(collocations)
meta_df = pd.DataFrame(is_metaphor)
src_df = pd.DataFrame(sources)
metaphors = coll_df.merge(meta_df, how='outer', on=['text_id', 'starts', 'ends']).merge(
    src_df, how='outer', on=['text_id', 'starts', 'ends'])
# берем только метафоры
metaphors = metaphors[metaphors['is_metaphor_Liza'] == True]

In [21]:
metaphors['meta_text'] = metaphors.apply(lambda x: merge_texts(x['text'], x['text_x'], x['text_y']), axis=1)
metaphors['is_collocation'].fillna(False, inplace=True)
metaphors.sort_values(['text_id', 'starts'], inplace=True)
metaphors = metaphors[[
    'text_id', 'starts', 'ends', 'meta_text', 'is_collocation',
    'is_metaphor_Masha', 'is_metaphor_Liza', 'is_metaphor_Olesia',
    'source_Masha', 'target_Masha', 'source_Liza',
    'target_Liza', 'source_Olesia', 'target_Olesia']]
print(metaphors.shape)
metaphors.head()

(356, 14)


,text_id,starts,ends,meta_text,is_collocation,is_metaphor_Masha,is_metaphor_Liza,is_metaphor_Olesia,source_Masha,target_Masha,source_Liza,target_Liza,source_Olesia,target_Olesia
38,1,29,38,выпустило,False,True,True,True,ФИЗИЧЕСКОЕ ДЕЙСТВИЕ,ПРОИЗВОДСТВО,ФИЗИЧЕСКОЕ ДЕЙСТВИЕ,ПРОИЗВОДСТВО,ФИЗИЧЕСКОЕ ДЕЙСТВИЕ,ПРОИЗВОДСТВО
41,1,259,267,вышедший,False,True,True,True,ДВИЖЕНИЕ,ПРОИЗВОДСТВО,ДВИЖЕНИЕ,ПРОИЗВОДСТВО,ДВИЖЕНИЕ,ПРОИЗВОДСТВО
0,1,443,455,не скажешь о,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN
42,1,503,511,проходит,False,True,True,True,ДВИЖЕНИЕ,СОБЫТИЕ,ДВИЖЕНИЕ,СОБЫТИЕ,ДВИЖЕНИЕ,СОБЫТИЕ
45,1,759,769,переводили,False,True,True,True,ДВИЖЕНИЕ,МЕНТАЛЬНОЕ ВОЗДЕЙСТВИЕ,ДВИЖЕНИЕ,ИЗМЕНЕНИЕ,ДВИЖЕНИЕ,МЕНТАЛЬНОЕ ВОЗДЕЙСТВИЕ


In [22]:
meta_dict = metaphors.to_dict(orient='records')
for meta in meta_dict:
    temp = sent_df[(sent_df['text_id'] == meta['text_id']) &
                   (sent_df['starts'] <= meta['starts']) &
                   (sent_df['ends'] >= meta['ends'])]
    meta['sent_id'] = temp['sent_id'].values[0]

In [23]:
metaphors = pd.DataFrame(meta_dict)
metaphors = pd.concat(
    [metaphors, metaphors.groupby(['text_id', 'sent_id']).cumcount().reset_index(drop=True)],
    axis=1)
metaphors.rename(columns={0: 'meta_num_in_sent'}, inplace=True)

In [24]:
def generate_id(parent_id, num_in_parent):
    return int(f'{parent_id}00{num_in_parent}')

In [25]:
metaphors['meta_id'] = metaphors.apply(lambda x: generate_id(x['sent_id'], x['meta_num_in_sent']), axis=1)

In [26]:
metaphors = metaphors[[
    'text_id', 'sent_id', 'meta_num_in_sent',
    'meta_id', 'starts', 'ends', 'meta_text',
    'is_collocation', 'source_Liza', 'target_Liza']]
metaphors.rename(columns={'source_Liza': 'source',
                          'target_Liza': 'target'}, inplace=True)
print(metaphors.shape)
metaphors.head()

(356, 10)


,text_id,sent_id,meta_num_in_sent,meta_id,starts,ends,meta_text,is_collocation,source,target
0,1,1000,0,1000000,29,38,выпустило,False,ФИЗИЧЕСКОЕ ДЕЙСТВИЕ,ПРОИЗВОДСТВО
1,1,1002,0,1002000,259,267,вышедший,False,ДВИЖЕНИЕ,ПРОИЗВОДСТВО
2,1,1003,0,1003000,443,455,не скажешь о,True,NaN,NaN
3,1,1003,1,1003001,503,511,проходит,False,ДВИЖЕНИЕ,СОБЫТИЕ
4,1,1004,0,1004000,759,769,переводили,False,ДВИЖЕНИЕ,ИЗМЕНЕНИЕ


### Токены

In [27]:
tok_df = pd.DataFrame(tokens)
tok_df.sort_values(['text_id', 'starts'], inplace=True)
print(tok_df.shape)
tok_df.head()

(12183, 9)


,text_id,starts,ends,text,lemma,pos,deprel,head_starts,head_ends
32,1,0,12,Издательство,издательство,NOUN,nsubj,29,38
33,1,13,14,«,"""",PUNCT,punct,15,17
34,1,15,17,Ад,Ад,PROPN,nmod,0,12
35,1,18,26,Маргинем,Маргин,PROPN,appos,15,17
36,1,27,28,»,"""",PUNCT,punct,15,17


In [28]:
tok_dict = tok_df.to_dict(orient='records')
for token in tok_dict:
    temp = sent_df[(sent_df['text_id'] == token['text_id']) &
                   (sent_df['starts'] <= token['starts']) &
                   (sent_df['ends'] >= token['ends'])]
    token['sent_id'] = temp['sent_id'].values[0]
    try:
        temp = metaphors[(metaphors['text_id'] == token['text_id']) &
                         (metaphors['starts'] <= token['starts']) &
                         (metaphors['ends'] >= token['ends'])]
        token['meta_id'] = temp['meta_id'].values[0]
    except IndexError:
        token['meta_id'] = None

In [29]:
tok_df = pd.DataFrame(tok_dict)
tok_df = pd.concat(
    [tok_df, tok_df.groupby(['text_id', 'sent_id', 'meta_id']).cumcount().reset_index(drop=True)],
    axis=1)
tok_df.rename(columns={0: 'tok_num_in_meta'}, inplace=True)

In [30]:
def generate_from_position(parent_id, start, end):
    return int(f'{parent_id}00{start}00{end}')

In [31]:
tok_df['tok_id'] = tok_df.apply(lambda x: generate_from_position(x['sent_id'], x['starts'], x['ends']), axis=1)

In [32]:
head_dict = tok_df.to_dict(orient='records')
for token in head_dict:
    temp = tok_df[(tok_df['text_id'] == token['text_id']) &
                  (tok_df['sent_id'] == token['sent_id']) &
                  (tok_df['starts'] == token['head_starts']) &
                  (tok_df['ends'] == token['head_ends'])]
    token['head_id'] = temp['tok_id'].values[0]

In [33]:
tok_df = pd.DataFrame(head_dict)
print(tok_df.shape)
tok_df.head()

(12183, 14)


,text_id,starts,ends,text,lemma,pos,deprel,head_starts,head_ends,sent_id,meta_id,tok_num_in_meta,tok_id,head_id
0,1,0,12,Издательство,издательство,NOUN,nsubj,29,38,1000,NaN,NaN,10000000012,100000290038
1,1,13,14,«,"""",PUNCT,punct,15,17,1000,NaN,NaN,100000130014,100000150017
2,1,15,17,Ад,Ад,PROPN,nmod,0,12,1000,NaN,NaN,100000150017,10000000012
3,1,18,26,Маргинем,Маргин,PROPN,appos,15,17,1000,NaN,NaN,100000180026,100000150017
4,1,27,28,»,"""",PUNCT,punct,15,17,1000,NaN,NaN,100000270028,100000150017


## Запись в базу
### Тексты

In [34]:
def escape_quotes(value):
    if isinstance(value, str):
        return value.replace("'", "\\'").replace('"', "\\'")
    return value

In [35]:
text_df['text'] = text_df['text'].map(escape_quotes)

In [36]:
for row in text_df.to_dict(orient='records'):
    vals = list(row.values())
    cur.execute("""
    INSERT INTO texts
    VALUES ("{}", "{}", "{}", "{}", NULLIF("{}", 'None'))""".format(*list(row.values())))
    con.commit()

### Предложения

In [37]:
sent_df['text'] = sent_df['text'].map(escape_quotes)

In [38]:
for row in sent_df.to_dict(orient='records'):
    cur.execute("""
    INSERT INTO sentences (text_id, id_sentences, sent_num_in_text, start, end, sent_text)
    VALUES ("{}", "{}", "{}", "{}", "{}", "{}")
    """.format(*list(row.values())))
    con.commit()

### Метафоры

In [39]:
metaphors['meta_text'] = metaphors['meta_text'].map(escape_quotes)
metaphors.replace(to_replace={False: 0, True: 1}, inplace=True)

In [40]:
for row in metaphors.to_dict(orient='records'):
    cur.execute("""
    INSERT INTO metaphors (text_id, sent_id, meta_num_in_sent, id_metaphors, start, end, meta_text, is_collocation, source, target)
    VALUES ("{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", NULLIF("{}", 'None'), NULLIF("{}", 'None'))
    """.format(*list(row.values())))
    con.commit()

### Токены

In [41]:
tok_df['text'] = tok_df['text'].map(escape_quotes)
tok_df['lemma'] = tok_df['lemma'].map(escape_quotes)
tok_df.replace(to_replace={np.nan: None}, inplace=True)

In [42]:
tok_to_insert = tok_df[['tok_id', 'text', 'tok_num_in_meta',
                        'starts', 'ends', 'pos', 'lemma',
                        'deprel', 'head_id', 'meta_id',
                        'sent_id', 'text_id']]
tok_to_insert

,tok_id,text,tok_num_in_meta,starts,ends,pos,lemma,deprel,head_id,meta_id,sent_id,text_id
0,10000000012,Издательство,None,0,12,NOUN,издательство,nsubj,100000290038,None,1000,1
1,100000130014,«,None,13,14,PUNCT,\',punct,100000150017,None,1000,1
2,100000150017,Ад,None,15,17,PROPN,Ад,nmod,10000000012,None,1000,1
3,100000180026,Маргинем,None,18,26,PROPN,Маргин,appos,100000150017,None,1000,1
4,100000270028,»,None,27,28,PUNCT,\',punct,100000150017,None,1000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12178,100058006639006648,перестали,None,6639,6648,VERB,перестать,advcl,100058006614006621,None,100058,10
12179,100058006649006651,на,None,6649,6651,ADP,на,case,100058006652006655,None,100058,10
12180,100058006652006655,ней,None,6652,6655,PRON,она,obl,100058006656006671,None,100058,10
12181,100058006656006671,останавливаться,None,6656,6671,VERB,останавливаться,xcomp,100058006639006648,None,100058,10


In [43]:
for row in tok_to_insert.to_dict(orient='records'):
    cur.execute("""
    INSERT INTO tokens
    VALUES ("{}", "{}", NULLIF("{}", 'None'), "{}", "{}", "{}", "{}", "{}", "{}", NULLIF("{}", 'None'), "{}", "{}")
    """.format(*list(row.values())))
    con.commit()

In [44]:
cur.execute("""
ALTER TABLE tokens
  ADD CONSTRAINT tok_to_head
    FOREIGN KEY (head)
    REFERENCES tokens (id_tokens)
    ON DELETE CASCADE
    ON UPDATE CASCADE;
""")
con.commit()

In [45]:
con.close()